In [17]:
import pandas as pd
import seaborn as sns
import numpy as np
import os.path as op
import glob
import matplotlib.pyplot as plt
import ast
# import moss
import csv
import random
%matplotlib inline
import statsmodels.api as sm
import statsmodels.formula.api as smf
from statsmodels.tools.sm_exceptions import ConvergenceWarning
import scipy
import statsmodels.api as sm
import statsmodels.formula.api as smf
from statsmodels.tools.sm_exceptions import ConvergenceWarning
import os
from pandas import Timestamp
from datetime import timedelta
from sklearn.preprocessing import StandardScaler


In [18]:
home_dir = op.abspath('./')

data_files = glob.glob(op.join(home_dir,
                        'data',
                        '*.csv'))
sns.set_context('talk')

In [19]:
column_name = 'image_recall_response.keys'
threshold = 0.4

In [20]:
creation_dates = []
for file_path in data_files:
    try:
        df = pd.read_csv(file_path)
        if column_name in df.columns:
            creation_date = df['date']
            creation_dates.append(creation_date)
    except Exception as e:
#             print(f"Error reading {file_path}: {e}")
        continue

In [21]:
def parse_dates(series):
    date_str = series.iloc[0]
    if "24h" in date_str:
        corrected_date_str = date_str.replace("24h", "00h")
        dt = pd.to_datetime(corrected_date_str, format='%Y-%m-%d_%Hh%M.%S.%f')
        dt += timedelta(days=1)
    else:
        dt = pd.to_datetime(date_str, format='%Y-%m-%d_%Hh%M.%S.%f')
    return dt

dates = [parse_dates(series) for series in creation_dates]

min_date = min(dates)


In [22]:
def remove_unit_variance(df, col, unit, group=None, suffix="_within"):
    """Remove variance between sampling units.

    This is useful for plotting repeated-measures data using within-unit
    error bars.

    Parameters
    ----------
    df : DataFrame
        Input data. Will have a new column added.
    col : column name
        Column in dataframe with quantitative measure to modify.
    unit : column name
        Column in dataframe defining sampling units (e.g., subjects).
    group : column name(s), optional
        Columns defining groups to remove unit variance within.
    suffix : string, optional
        Suffix appended to ``col`` name to create new column.

    Returns
    -------
    df : DataFrame
        Returns modified dataframe.

    """
    new_col = col + suffix

    def demean(x):
        return x - x.mean()

    if group is None:
        new = df.groupby(unit)[col].transform(demean)
        new += df[col].mean()
        df.loc[:, new_col] = new
    else:
        df.loc[:, new_col] = np.nan
        for level, df_level in df.groupby(group):
            new = df_level.groupby(unit)[col].transform(demean)
            new += df_level[col].mean()
            df.loc[new.index, new_col] = new

    return df


In [23]:
def df_demean(df, list_of_variables):
    for l in list_of_variables:
        df[l] = df[l] - np.mean(df[l])
        z_scored = f"{l}_z"
        df[z_scored] = scaler.fit_transform(df[[l]])

    return df

In [24]:
def df_square_and_mean(df, list_of_variables):
    for l in list_of_variables:
        squared_col = f"{l}_sq"
        df[squared_col] = df[l]**2
        df[squared_col] = df[squared_col] - np.mean(df[squared_col])
        z_scored = f"{squared_col}_z"
        df[z_scored] = scaler.fit_transform(df[[squared_col]])
    return df




In [25]:
def convert_string(value):
    try:
        # Check if value is a string and looks like a list
        if isinstance(value, str) and value.startswith('[') and value.endswith(']'):
            # Remove brackets and split by comma
            cleaned = value.strip('[]')
            if cleaned:  # Check if the string is not empty
                # Convert each part to float and calculate average
                numbers = [float(num) for num in cleaned.split(',')]
                return sum(numbers) / len(numbers)
            else:
                return None  # Return None or another placeholder for empty lists
        else:
            return None
    except Exception as e:
        print(f"Error converting value {value}: {e}")
        return None


In [26]:
def calculate_differences(dataframe):
    # Determine attended and unattended values for IT and V2
    dataframe['it_sim_dis_attend'] = np.where(dataframe['attend'] == 'img1', dataframe['IT_root_im1'], dataframe['IT_root_im2'])
    dataframe['v2_sim_dis_attend'] = np.where(dataframe['attend'] == 'img1', dataframe['V2_root_im1'], dataframe['V2_root_im2'])

    dataframe['it_sim_dis_test'] = np.where(dataframe['test_item'] == 'img1', dataframe['IT_root_im1'], dataframe['IT_root_im2'])
    dataframe['v2_sim_dis_test'] = np.where(dataframe['test_item'] == 'img1', dataframe['V2_root_im1'], dataframe['V2_root_im2'])

    
    dataframe['it_sim_dis_unattend'] = np.where(dataframe['attend'] != 'img1', dataframe['IT_root_im1'], dataframe['IT_root_im2'])
    dataframe['v2_sim_dis_unattend'] = np.where(dataframe['attend'] != 'img1', dataframe['V2_root_im1'], dataframe['V2_root_im2'])

    
    dataframe['it_sim_dis_untest'] = np.where(dataframe['test_item'] != 'img1', dataframe['IT_root_im1'], dataframe['IT_root_im2'])
    dataframe['v2_sim_dis_untest'] = np.where(dataframe['test_item'] != 'img1', dataframe['V2_root_im1'], dataframe['V2_root_im2'])

    # Calculate differences
    dataframe['it_sim_dis_diff'] = np.where(dataframe['attend'] == 'img1', dataframe['IT_root_im1'] - dataframe['IT_root_im2'], dataframe['IT_root_im2'] - dataframe['IT_root_im1'])
    dataframe['v2_sim_dis_diff'] = np.where(dataframe['attend'] == 'img1', dataframe['V2_root_im1'] - dataframe['V2_root_im2'], dataframe['V2_root_im2'] - dataframe['V2_root_im1'])

    dataframe['it_sim_dis_diff_test'] = np.where(dataframe['test_item'] == 'img1', dataframe['IT_root_im1'] - dataframe['IT_root_im2'], dataframe['IT_root_im2'] - dataframe['IT_root_im1'])
    dataframe['v2_sim_dis_diff_test'] = np.where(dataframe['test_item'] == 'img1', dataframe['V2_root_im1'] - dataframe['V2_root_im2'], dataframe['V2_root_im2'] - dataframe['V2_root_im1'])

    # Determine convergence and preferences
    dataframe['v2_converges'] = np.where((dataframe['it_sim_dis_diff'] > 0) & (dataframe['v2_sim_dis_diff'] > 0) | (dataframe['it_sim_dis_diff'] < 0) & (dataframe['v2_sim_dis_diff'] < 0), 'V2/IT agree', 'V2/IT disagree')

    dataframe['v2_prefers'] = np.where(dataframe['v2_sim_dis_diff'] > 0, 'Prioritized', 'Deprioritized')
    dataframe['it_prefers'] = np.where(dataframe['it_sim_dis_diff'] > 0, 'Prioritized', 'Deprioritized')
    
    
    dataframe['v2_prefers_test'] = np.where(dataframe['v2_sim_dis_diff_test'] > 0, 'Tested', 'Untested')
    dataframe['it_prefers_test'] = np.where(dataframe['it_sim_dis_diff_test'] > 0, 'Tested', 'Untested')
    
    
    dataframe['Distractor V2 Similarity Preference Tested'] = dataframe['v2_prefers_test'] 
    dataframe['Distractor IT Similarity Preference Tested'] = dataframe['it_prefers_test']
    
    
    
    dataframe['Distractor V2 Similarity Preference'] = dataframe['v2_prefers'] 
    dataframe['Distractor IT Similarity Preference'] = dataframe['it_prefers']
    
    dataframe['it_im1_im2'] = dataframe['IT_im1_im2']
    dataframe['v2_im1_im2'] = dataframe['V2_im1_im2']
    
    dataframe['IT_diff_binned'] = pd.qcut(dataframe['it_sim_dis_diff'], 5, duplicates='drop')
    dataframe['V2_diff_binned'] = pd.qcut(dataframe['v2_sim_dis_diff'], 5, duplicates='drop')
    
    dataframe['IT_diff_binned_test'] = pd.qcut(dataframe['it_sim_dis_diff_test'], 5, duplicates='drop')
    dataframe['V2_diff_binned_test'] = pd.qcut(dataframe['v2_sim_dis_diff_test'], 5, duplicates='drop')
    
    
    dataframe = df_demean(dataframe, ['it_sim_dis_diff', 'v2_sim_dis_diff', 'it_sim_dis_attend', 'v2_sim_dis_attend', 'it_sim_dis_unattend', 'v2_sim_dis_unattend', 'it_sim_dis_test', 'v2_sim_dis_test', 'it_sim_dis_untest', 'v2_sim_dis_untest', 'it_sim_dis_diff_test', 'v2_sim_dis_diff_test' ])

    dataframe = df_square_and_mean(dataframe, ['it_sim_dis_diff', 'v2_sim_dis_diff', 'it_sim_dis_attend', 'v2_sim_dis_attend', 'it_sim_dis_unattend', 'v2_sim_dis_unattend', 'it_sim_dis_test', 'v2_sim_dis_test', 'it_sim_dis_untest', 'v2_sim_dis_untest', 'it_sim_dis_diff_test', 'v2_sim_dis_diff_test' ])

    
    
    return dataframe

In [27]:
def categorize_columns(dataframe, column_params):
    """
    Categorizes specified columns in a DataFrame into discrete categories based on quantiles.

    Parameters:
    - dataframe: The DataFrame to process.
    - column_params: A dictionary specifying the number of categories and labels for each column prefix.

    Returns:
    - The modified DataFrame with additional columns for categorized data.
    """
    for label in ['it_sim_dis_attend', 'v2_sim_dis_attend', 'it_sim_dis_unattend', 'v2_sim_dis_unattend', 'it_sim_dis_diff', 'v2_sim_dis_diff', 'it_im1_im2', 'v2_im1_im2', 'it_sim_dis_test', 'v2_sim_dis_test', 'it_sim_dis_untest', 'v2_sim_dis_untest', 'it_sim_dis_diff_test', 'v2_sim_dis_diff_test', 'v2_sim_dis_diff_sq', 'it_sim_dis_diff_sq', 'v2_sim_dis_diff_test_sq','it_sim_dis_diff_test_sq']:
        # Determine the column prefix to decide which set of parameters to use
        column_prefix = 'v2' if 'v2' in label else 'it'
        
        # Extract the number of categories and labels from the parameters dictionary
        n_cats = column_params[column_prefix]['n_cats']
        labels = column_params[column_prefix]['labels']
        
        # Categorize the column data
        dataframe[label + '_cat'] = pd.qcut(dataframe[label], q=n_cats, labels=labels, duplicates='drop')
    
    return dataframe

In [28]:
def validity_assignment(df):
    x = []
    for i in df['validity']:
        if i == 'valid':
            x.append('prioritized')
        else:
            x.append('deprioritized')

    df['Tested Item'] = x

In [29]:
def df_column_addition(df):
    
    df['V2 Distractor Similarity\nto Prioritized Item'] = df['v2_sim_dis_attend_cat']
    df['IT Distractor Similarity\nto Prioritized Item'] = df['it_sim_dis_attend_cat']
    df['V2 Distractor Similarity\nto Deprioritized Item'] = df['v2_sim_dis_unattend_cat']
    df['IT Distractor Similarity\nto Deprioritized Item'] = df['it_sim_dis_unattend_cat']
    df['Prioritized - Deprioritized IT Distractor Similarity'] = df['it_sim_dis_diff_cat'] 
    df['Prioritized - Deprioritized V2 Distractor Similarity'] = df['v2_sim_dis_diff_cat'] 
    df['Prioritized - Deprioritized IT Distractor Similarity Squared'] = df['v2_sim_dis_diff_sq_cat'] 
    df['Prioritized - Deprioritized V2 Distractor Similarity Squared'] = df['it_sim_dis_diff_sq_cat'] 

    
    df['V2 Distractor Similarity\nto Tested Item'] = df['v2_sim_dis_test_cat']
    df['IT Distractor Similarity\nto Tested Item'] = df['it_sim_dis_test_cat']
    df['V2 Distractor Similarity\nto Untested Item'] = df['v2_sim_dis_untest_cat']
    df['IT Distractor Similarity\nto Untested Item'] = df['it_sim_dis_untest_cat']
    df['Tested - Untested IT Distractor Similarity'] = df['it_sim_dis_diff_test_cat'] 
    df['Tested - Untested V2 Distractor Similarity'] = df['v2_sim_dis_diff_test_cat'] 
    df['Tested - Untested IT Distractor Similarity Squared'] = df['v2_sim_dis_diff_test_sq_cat'] 
    df['Tested - Untested V2 Distractor Similarity Squared'] = df['it_sim_dis_diff_test_sq_cat'] 

   
    df['IT_diff_binned'] = pd.qcut(df['it_sim_dis_diff'], 5, duplicates='drop')
    df['V2_diff_binned'] = pd.qcut(df['v2_sim_dis_diff'], 5, duplicates='drop')
    df['IT_diff_binned_sq'] = pd.qcut(df['it_sim_dis_diff_sq'], 5, duplicates='drop')
    df['V2_diff_binned_sq'] = pd.qcut(df['v2_sim_dis_diff_sq'], 5, duplicates='drop')


    df['IT_diff_binned_test'] = pd.qcut(df['it_sim_dis_diff_test'], 5, duplicates='drop')
    df['V2_diff_binned_test'] = pd.qcut(df['v2_sim_dis_diff_test'], 5, duplicates='drop')
    df['IT_diff_binned_test_sq'] = pd.qcut(df['it_sim_dis_diff_test_sq'], 5, duplicates='drop')
    df['V2_diff_binned_test_sq'] = pd.qcut(df['v2_sim_dis_diff_test_sq'], 5, duplicates='drop')

    def round_to_significant_figures(x, sig_figs=2):
        if x == 0:
            return 0
        else:
            return round(x, sig_figs - int(np.floor(np.log10(abs(x)))) - 1)


    def process_interval(interval):
        # Round both bounds
        lower_rounded = round_to_significant_figures(interval.left)
        upper_rounded = round_to_significant_figures(interval.right)
        # Return a new interval with the rounded bounds
        return pd.Interval(lower_rounded, upper_rounded)

    df['IT_diff_binned'] = df['IT_diff_binned'].apply(process_interval)
    df['V2_diff_binned'] = df['V2_diff_binned'].apply(process_interval)
    df['IT_diff_binned_sq'] = df['IT_diff_binned_sq'].apply(process_interval)
    df['V2_diff_binned_sq'] = df['V2_diff_binned_sq'].apply(process_interval)

    df['IT_diff_binned_test'] = df['IT_diff_binned_test'].apply(process_interval)
    df['V2_diff_binned_test'] = df['V2_diff_binned_test'].apply(process_interval)
    df['IT_diff_binned_test_sq'] = df['IT_diff_binned_test_sq'].apply(process_interval)
    df['V2_diff_binned_test_sq'] = df['V2_diff_binned_test_sq'].apply(process_interval)

    df['Prioritized - Deprioritized V2 Distractor Similarity Ranges'] = df['V2_diff_binned']
    df['Prioritized - Deprioritized IT Distractor Similarity Ranges'] = df['IT_diff_binned']
    df['Prioritized - Deprioritized V2 Distractor Similarity Squared Ranges'] = df['V2_diff_binned_sq']
    df['Prioritized - Deprioritized IT Distractor Similarity Squared Ranges'] = df['IT_diff_binned_sq']

    df['Tested - Untested V2 Distractor Similarity Ranges'] = df['V2_diff_binned_test']
    df['Tested - Untested IT Distractor Similarity Ranges'] = df['IT_diff_binned_test']
    df['Tested - Untested V2 Distractor Similarity Squared Ranges'] = df['V2_diff_binned_test_sq']
    df['Tested - Untested IT Distractor Similarity Squared Ranges'] = df['IT_diff_binned_test_sq']
    
    
    df['tested_item'] = df['Tested Item']
    df['ret_rel'] = df['Retrocue Reliability']
    
    df['validity_binary'] = df['Tested Item'].apply(lambda x: 1 if x == 'prioritized' else 0)
    df['reliability_binary'] = df['Retrocue Reliability'].apply(lambda x: 1 if x == 'high' else 0)
    df['validity_binary_z'] = scaler.fit_transform(df[['validity_binary']])
    df['reliability_binary_z'] = scaler.fit_transform(df[['reliability_binary']])



    
    df['V2 Distractor Similarity to Tested Item'] = df['v2_sim_dis_test_z']
    df['IT Distractor Similarity to Tested Item'] = df['it_sim_dis_test_z']
    df['Tested - Untested V2 Distractor Similarity'] = df['v2_sim_dis_diff_test_z']
    df['Tested - Untested IT Distractor Similarity'] = df['it_sim_dis_diff_test_z']


    df['V2 Distractor Similarity\nto Prioritized Item'] = df['v2_sim_dis_attend_z']
    df['IT Distractor Similarity\nto Prioritized Item'] = df['it_sim_dis_attend_z']
    df['V2 Distractor Similarity\nto Deprioritized Item'] = df['v2_sim_dis_unattend_z']
    df['IT Distractor Similarity\nto Deprioritized Item'] = df['it_sim_dis_unattend_z']

    df['Prioritized - Deprioritized IT Distractor Similarity'] = df['it_sim_dis_diff_z'] 
    df['Prioritized - Deprioritized V2 Distractor Similarity'] = df['v2_sim_dis_diff_z'] 
    


In [30]:
def df_creation(data_files, start_date, end_date):
    processed_dfs = []
    date_column = 'date'

    for file_path in data_files:
        try:
            # Read the CSV file and append to the list of DataFrames
            temp_df = pd.read_csv(file_path)
            temp_df['filename'] = file_path
            processed_dfs.append(temp_df)
        except Exception as e:
            # Optionally, print or log the error
#             print(f"Failed to read {file_path}: {e}")
            continue

    if processed_dfs:
        df = pd.concat(processed_dfs, ignore_index=True)
        df[date_column] = pd.to_datetime(df[date_column], format='%Y-%m-%d_%Hh%M.%S.%f', errors='coerce')
        df.dropna(subset=[date_column], inplace=True)
        df = df[(df[date_column] >= pd.to_datetime(start_date)) & (df[date_column] <= pd.to_datetime(end_date))]
        
        # # Uncomment and adapt the following lines as needed:
        df = df.loc[df['V2_diff'].notnull()].reset_index(drop=True)
        df['reliability'] = df['reliability'].astype(float)
        df['Retrocue Reliability'] = np.where(df['reliability'] > 0.75, 'high', 'low')
        non_numeric_values = df['resp_correct'][~df['resp_correct'].apply(np.isreal)]
        numeric_columns = df.select_dtypes(include=['float64', 'int64']).columns
        result = df[numeric_columns].groupby('participant').mean()
        
        

    else:
        # Return an empty DataFrame if no files were processed successfully
        df = pd.DataFrame()
    
    return df, numeric_columns


   

In [35]:
df, numeric_columns = df_creation(data_files,'2024-10-30', '2024-11-22')
### pilot5 '2024-10-08', '2024-10-30'
### pilot6 '2024-11-22', '2025-01-30'

In [36]:
df['pilot_number'] = np.where(df['date'] <= '2024-10-31', 1, 2)
df_1 = df[df['pilot_number'] == 1]
df_2 = df[df['pilot_number'] == 2]

len(set(df_1['participant'])), len(set(df_2['participant']))

(0, 4)

In [ ]:
def add_trial_info(df, participant_col, trials_per_batch=30):
    
    df['Trial_Number'] = df.groupby(participant_col).cumcount() + 1
    
    df['Trial_Batch'] = ((df['Trial_Number'] - 1) // trials_per_batch) + 1
    
    return df

add_trial_info(df, participant_col='participant')  

In [ ]:
df_trial_182 = df[df['trial'] == 182]
df_trial_182 = df_trial_182[['participant', 'trial', 'cond_file', 'root', 'IT_diff']]
df_trial_182['conditions_batch'] = df_trial_182.groupby(['cond_file', 'root', 'IT_diff']).ngroup() + 1

batch_info = df_trial_182.groupby('conditions_batch')['participant'].agg(
    participants_count='nunique',  # Count of unique participants
    participants_list='unique'  # List of participants in the batch
).reset_index()

df_trial_182 = df_trial_182.merge(batch_info, on='conditions_batch', how='left')



In [ ]:
df = df.merge(df_trial_182[['participant', 'conditions_batch', 'participants_count', 'participants_list']], 
                     on='participant', 
                     how='left')


In [ ]:
df['date'] = pd.to_datetime(df['date'], errors='coerce')


In [ ]:
column_params = {
    'v2': {'n_cats': 5, 'labels': ['Least Similar', '', '  ', '   ', 'Most Similar']},
    'it': {'n_cats': 5, 'labels': ['Least Similar', '', '  ', '   ', 'Most Similar']}
}
column_name = 'image_recall_response.keys'
threshold = 0.4

In [ ]:
scaler = StandardScaler()


In [ ]:
validity_assignment(df)
df = calculate_differences(df)
df = categorize_columns(df, column_params)
df_column_addition(df)
# label_addition(df)

In [ ]:
df['reliability_binary'] = (df['reliability'] > 0.7).astype(int)
df['validity_binary'] = (df['validity'] == 'valid').astype(int)


In [ ]:


def process_values_time(value):
    try:
        # Strip square brackets and split by comma, filtering out empty values
        if isinstance(value, str) and value.startswith('[') and value.endswith(']'):
            cleaned = value.strip('[]').split(',')
            if cleaned == ['']:  # Check if the list after stripping is empty
                return None
            numbers = [float(num) for num in cleaned if num.strip()]
            return numbers
        return None
    except ValueError:
        return None

def process_values_click(value):
    try:
        # Strip square brackets and split by comma, filtering out empty values
        if isinstance(value, str) and value.startswith('[') and value.endswith(']'):
            cleaned = value.strip('[]').split(',')
            if cleaned == ['']:  # Check if the list after stripping is empty
                return None
            numbers = [num for num in cleaned if num.strip()]
            return numbers
        return None
    except ValueError:
        return None

    
df['processed_mouse.time'] = df['mouse.time'].apply(process_values_time)
df['processed_mouse.click'] = df['mouse.clicked_name'].apply(process_values_click)

def get_latest_value(entry):
    if isinstance(entry, list) and len(entry) > 1:
        return [entry[-1]]  # Return a list with the latest value
    return entry  # Return unchanged if not a list or list is empty/single element

df['processed_mouse.time'] = df['processed_mouse.time'].apply(get_latest_value)



In [ ]:

df['date'] = pd.to_datetime(df['date'], errors='coerce')

# Find the earliest date for each participant
earliest_dates = df.groupby('participant')['date'].min().reset_index()

df = pd.merge(df, earliest_dates, on=['participant', 'date'])



participant_summary = df.groupby('participant').agg(
    total_entries=pd.NamedAgg(column='participant', aggfunc='size')
).reset_index()

participant_file_counts = df.groupby(['participant', 'filename']).agg(
    entry_count=pd.NamedAgg(column='participant', aggfunc='size'),
    
    none_mouse=pd.NamedAgg(column='processed_mouse.time', aggfunc=lambda x: x.isna().sum())
).reset_index()



files_with_high_none_mouse = participant_file_counts[participant_file_counts['none_mouse'] > 100]

df_cleaned = df[~df.set_index(['participant', 'filename']).index.isin(files_with_high_none_mouse.set_index(['participant', 'filename']).index)]

participant_file_counts_cleaned = df_cleaned.groupby(['participant', 'filename']).agg(
    entry_count=pd.NamedAgg(column='participant', aggfunc='size')
).reset_index()

files_with_low_entries = participant_file_counts_cleaned[participant_file_counts_cleaned['entry_count'] < 200]

df_cleaned = df_cleaned[~df_cleaned.set_index(['participant', 'filename']).index.isin(files_with_low_entries.set_index(['participant', 'filename']).index)]

cleaned_participant_summary = df_cleaned.groupby('participant').agg(
    total_entries=pd.NamedAgg(column='participant', aggfunc='size')
).reset_index()


none_mouse_summary = df_cleaned.groupby('participant').agg(
    none_mouse=pd.NamedAgg(column='processed_mouse.time', aggfunc=lambda x: x.isna().sum())
).reset_index()

df = df_cleaned

In [ ]:
duplicate_participants = df.groupby('participant')['filename'].nunique().reset_index()
duplicate_participants = duplicate_participants[duplicate_participants['filename'] > 1]

if not duplicate_participants.empty:
    print("\nParticipants associated with more than one file:")
    print(duplicate_participants)
else:
    print("\nNo participants associated with more than one file.")

In [ ]:
none_mouse_summary = df.groupby('participant').agg(
    none_mouse=pd.NamedAgg(column='processed_mouse.time', aggfunc=lambda x: x.isna().sum())
).reset_index()

# Check for participants with none_mouse more than 2/3 of total_entries
none_mouse_summary = none_mouse_summary.merge(cleaned_participant_summary, on='participant')
none_mouse_summary['ratio'] = none_mouse_summary['none_mouse'] / none_mouse_summary['total_entries']
participants_high_none_mouse = none_mouse_summary[none_mouse_summary['ratio'] > (2/3)]

# Print the results
print("Remaining participants with more than 300 entries:")
# print(cleaned_participant_summary)

print("\nNone mouse summary for each participant:")
# print(none_mouse_summary)

if not participants_high_none_mouse.empty:
    print("\nParticipants with none_mouse more than 2/3 of total entries:")
#     print(participants_high_none_mouse)
else:
    print("\nNo participants have none_mouse more than 2/3 of total entries.")

In [ ]:
participant_counts = df['participant'].value_counts()
participants_less_than_300 = participant_counts[participant_counts < 300]
participants_less_than_300


In [ ]:
def add_length_column(df, column_name):
    if column_name in df.columns:
        # Clean and convert the values in the specified column
        def clean_and_get_length(value):
            if isinstance(value, str) and value.startswith('[') and value.endswith(']'):
                cleaned = value.strip('[]').split(',')
                return len(cleaned)
            elif isinstance(value, list):
                return 0
            else:
                return 0  # Handle unexpected data types

        # Apply the cleaning and length calculation
        df.loc[:, column_name + '_length'] = df[column_name].apply(clean_and_get_length)
    else:
        print(f"Column '{column_name}' not found in DataFrame.")
    return df

def add_nthresp_column(df, column_name, n):
    if column_name in df.columns:
        # Clean and convert the values in the specified column
        def clean_and_get_nthresp(value):
            if isinstance(value, str) and value.startswith('[') and value.endswith(']'):
                cleaned = value.strip('[]').split(',')
                return cleaned[n]
            elif isinstance(value, list):
                return 
            else:
                return  # Handle unexpected data types

        # Apply the cleaning and length calculation
        if n == 0:
            temp_str = '_first'
        elif n == -1:
            temp_str = '_last'
        df.loc[:, column_name + temp_str] = df[column_name].apply(clean_and_get_nthresp)
    else:
        print(f"Column '{column_name}' not found in DataFrame.")
    return df

add_length_column(df, 'mouse.time')
add_nthresp_column(df, 'mouse.time', 0)
add_nthresp_column(df, 'mouse.time', -1)
add_nthresp_column(df, 'mouse.clicked_name', 0)
add_nthresp_column(df, 'mouse.clicked_name', -1)

# long_instances = df[df['mouse.time_length'] > 1]
# print(long_instances)
# unique_mouse_time_length_values = df['mouse.time_length'].unique()
# unique_mouse_time_length_values

In [ ]:
if 'participant' in df.columns and not df['participant'].isnull().all():
    print("'participant' column exists and has values.")
else:
    raise KeyError("The 'participant' column is missing or has no values.")

In [ ]:
def clean_dataframe_from_nonresp(df):
    # Convert the column entries from string lists to actual lists of floats, or None for empty lists

    # Define a mask to filter out rows where the list is empty or any number is out of the specified range
    def filter_ranges(numbers):
        if numbers is None:
            return False
        return all(num for num in numbers)
    df = df[df['processed_mouse.time'].apply(filter_ranges)]
    df = df[df['processed_mouse.click'].apply(filter_ranges)]
    # Apply the mask
    return df

In [ ]:
def df_with_threshold(df, numeric_columns, threshold):
    sub_df = df[numeric_columns].groupby('participant').mean().reset_index()
    above04_subs = sub_df.loc[sub_df['resp_correct'] >= threshold, 'participant']
    df_ret = pd.DataFrame()
    for s in above04_subs:
        df_ret = pd.concat([df_ret, df.loc[df['participant'] == s]])
    df_ret = remove_unit_variance(df_ret,'resp_correct','participant')
    df_ret['Accuracy'] = df_ret['resp_correct_within']
    return df_ret

df = df_with_threshold(df, numeric_columns, 0.4)

In [ ]:
df_nonresp = clean_dataframe_from_nonresp(df)

participant_counts = df_nonresp['participant'].value_counts()
participant_counts_df = participant_counts.reset_index()
participant_counts_df.columns = ['participant', 'count']

print(min(participant_counts_df['count']))

In [ ]:
numeric_columns = df.select_dtypes(include=['float64', 'int64']).columns


In [ ]:
df['mouse.time_last'] = pd.to_numeric(df['mouse.time_last'], errors='coerce')


In [ ]:
df = remove_unit_variance(df,'mouse.time_last','participant')


In [ ]:
df['analysis_rt'] = df['mouse.time_last_within']

In [ ]:
from collections import defaultdict



In [ ]:

df_table = defaultdict(int)
df_table_totals = defaultdict(int)

def update_count(row):
    if row['test_item'] == 'img1':
        df_table_totals[row['img1']] += 1
        if row['resp_correct'] == 1.0:
            df_table[row['img1']] += 1
    elif row['test_item'] == 'img2':
        df_table_totals[row['img2']] += 1
        if row['resp_correct'] == 1.0:
            df_table[row['img2']] += 1
        
df.apply(update_count, axis=1)
df_table = dict(df_table)


df_table_fraction = defaultdict(int)

for d in df_table.keys():
    df_table_fraction[d] = df_table[d]/df_table_totals[d]

In [ ]:
str(df['img1'])

In [ ]:
# df_table_fraction[]

In [ ]:
# df['attended_memorability'] = np.where(df['attend'] == 'img1', df_table_fraction[str(df['img1'])], df_table_fraction[str(df['img2'])])
# df['unattended_memorability'] = np.where(df['attend'] == 'img2', df_table_fraction[str(df['img2'])], df_table_fraction[str(df['img1'])])

# df['tested_memorability'] = np.where(df['test_item'] == 'img1', df_table_fraction[str(df['img1'])], df_table_fraction[str(df['img2'])])
# df['untested_memorability'] = np.where(df['test_item'] == 'img2', df_table_fraction[str(df['img2'])], df_table_fraction[str(df['img1'])])


In [ ]:
# df['tested_memorability'] = df['test_item'].map(lambda x: df_table_fraction[str(df['img1'])] if x == 'img1' else df_table_fraction[str(df['img2'])])
# df['untested_memorability'] = df['test_item'].map(lambda x: df_table_fraction[str(df['img2'])] if x == 'img1' else df_table_fraction[str(df['img1'])])


# df['attended_memorability'] = df['attend'].map(lambda x: df_table_fraction[str(df['img1'])] if x == 'img1' else df_table_fraction[str(df['img2'])])
# df['unattended_memorability'] = df['attend'].map(lambda x: df_table_fraction[str(df['img2'])] if x == 'img1' else df_table_fraction[str(df['img1'])])


In [ ]:
df['tested_memorability'] = np.where(
    df['test_item'] == 'img1',
    df['img1'].astype(str).map(df_table_fraction),
    df['img2'].astype(str).map(df_table_fraction)
)

df['untested_memorability'] = np.where(
    df['test_item'] == 'img1',
    df['img2'].astype(str).map(df_table_fraction),
    df['img1'].astype(str).map(df_table_fraction)
)

df['attended_memorability'] = np.where(
    df['attend'] == 'img1',
    df['img1'].astype(str).map(df_table_fraction),
    df['img2'].astype(str).map(df_table_fraction)
)

df['unattended_memorability'] = np.where(
    df['attend'] == 'img1',
    df['img2'].astype(str).map(df_table_fraction),
    df['img1'].astype(str).map(df_table_fraction)
)


In [ ]:
# df_memorability = pd.read_csv('predictions_pilot5.csv')


In [ ]:
# df_memorability['filename'] = df_memorability['filename'].str.replace('new_stimuli', 'stimuli')


In [ ]:
# df_memorability

In [ ]:
# memorability_dict = dict(zip(df_memorability['filename'], df_memorability['predictions']))


In [ ]:
# memorability_dict

In [ ]:
# df['tested_memorability_resmem'] = np.where(
#     df['test_item'] == 'img1',
#     df['img1'].astype(str).map(memorability_dict),
#     df['img2'].astype(str).map(memorability_dict)
# )

# df['untested_memorability_resmem'] = np.where(
#     df['test_item'] == 'img1',
#     df['img2'].astype(str).map(memorability_dict),
#     df['img1'].astype(str).map(memorability_dict)
# )

# df['attended_memorability_resmem'] = np.where(
#     df['attend'] == 'img1',
#     df['img1'].astype(str).map(memorability_dict),
#     df['img2'].astype(str).map(memorability_dict)
# )

# df['unattended_memorability_resmem'] = np.where(
#     df['attend'] == 'img1',
#     df['img2'].astype(str).map(memorability_dict),
#     df['img1'].astype(str).map(memorability_dict)
# )


# df['distractor_memorability'] = df['ping_img'].astype(str).map(memorability_dict)



In [ ]:
df['pilot_number'] = np.where(df['date'] <= '2024-10-31', 1, 2)


In [ ]:
df_1 = df[df['pilot_number'] == 1]

In [ ]:
len(set(df_1['participant']))

<!-- # df['tested_memorability_resmem_z'] = scaler.fit_transform(df[['tested_memorability_resmem']]) -->

In [ ]:
df['it_pos_neg'] =  np.where(df['it_sim_dis_diff_test_z'] <= 0, 0, 1)
df['v2_pos_neg'] =  np.where(df['v2_sim_dis_diff_test_z'] <= 0, 0, 1)
df['it_pos_neg_abs'] =  np.where(df['it_sim_dis_test_z'] <= 0, 0, 1)
df['v2_pos_neg_abs'] =  np.where(df['v2_sim_dis_test_z'] <= 0, 0, 1)

In [ ]:
df['rt'] = df['mouse.time_last']

In [ ]:
len(set(df['participant']))

In [ ]:
df['it_pos_neg_u'] =  np.where(df['it_sim_dis_diff_test'] <= 0, 0, 1)
df['v2_pos_neg_u'] =  np.where(df['v2_sim_dis_diff_test'] <= 0, 0, 1)
df['it_pos_neg_abs_u'] =  np.where(df['it_sim_dis_test'] <= 0, 0, 1)
df['v2_pos_neg_abs_u'] =  np.where(df['v2_sim_dis_test'] <= 0, 0, 1)

In [ ]:
df['it_sim_dis_diff_test']

In [ ]:
df['it_sim_dis_diff_test_z']

In [ ]:
# df1 = pd.read_csv('pilot5_TEST.csv')
# df2 = pd.read_csv('pilot6_TEST.csv')

# Concatenate the dataframes
# combined_df = pd.concat([df1, df2], ignore_index=True)

# Save to a new CSV if needed
# df.to_csv('pilot_TOTAL_TEST_edits.csv', index=False)

In [ ]:
# df_1 = combined_df[combined_df['pilot_number'] == 1]

In [ ]:
# df = combined_df

In [ ]:
# con = sqlite3.connect("pilot5_TEST.csv")
# cur = con.cursor()